<a href="https://colab.research.google.com/github/erinmurphy11/BOLD-LLM-Bias/blob/main/model_generation_Llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install datasets transformers evaluate -q
! pip install tensorflow
! pip install torch torchvision
! pip install matplotlib
! pip install tf-keras
! pip install scikit-learn
! pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!git clone https://github.com/erinmurphy11/BOLD-LLM-Bias.git

Cloning into 'BOLD-LLM-Bias'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 85 (delta 35), reused 57 (delta 16), pack-reused 0
Receiving objects: 100% (85/85), 1.68 MiB | 5.44 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [4]:
from datasets import load_from_disk
from transformers import pipeline, AutoTokenizer
import tqdm

# Read the BOLD JSON files into a dataset
load_path='/content/BOLD-LLM-Bias/prompts/sample_bold_prompts'
bold_prompts = load_from_disk(load_path)

# Load text generation pipeline and model
model = "daryl149/llama-2-7b-chat-hf"
text_generation = pipeline("text-generation", model=model)
tokenizer = AutoTokenizer.from_pretrained(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [5]:
def generate_text(prompts, max_length=100, do_sample=False, pad_token_id=502256):
	model_continuations=[]
	# Iterate through prompts and generate text
	for prompt in tqdm.tqdm(prompts):
		generation = text_generation(prompt,
									 max_length=max_length,
									 do_sample=do_sample,
									 pad_token_id=pad_token_id,
									 truncation=True)
		continuation = generation[0]['generated_text'].replace(prompt,'').lstrip()
		model_continuations.append(continuation)
	return model_continuations

In [6]:
# Select only first instance of prompt, some prompts have multiple sentences
toxic_prompts = [p['prompts'] for p in bold_prompts]

# Generate text with LLM using prompts
model_continuations = generate_text(toxic_prompts)

print('Generated '+ str(len(model_continuations))+ ' continuations')

# Update dataset with generations
bold_prompts = bold_prompts.add_column(f'generated_prompts_{model}', model_continuations)

100%|██████████| 100/100 [1:30:31<00:00, 54.31s/it]

Generated 100 continuations


In [9]:
# Save resulting prompts to disk
model = "meta-llama/Llama-2-7b-chat-hf"
out_name = f'./BOLD-LLM-Bias/generated_data/prompts_{model}'
bold_prompts.save_to_disk(out_name)

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

In [14]:
import shutil
from google.colab import files

directory_path = "/content/generated_data"
zip_file_path = "/content/generated_data.zip"
shutil.make_archive('/content/generated_data', 'zip', directory_path)
files.download(zip_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>